<a href="https://colab.research.google.com/github/juyoung524/22-1-Spring-/blob/main/20220323.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

다른 데 깔려있는 서비스를 API라고 해서 인터페이스 다른 서비스에 해주고 있을 때 개발자가 다른 서버나 데이터베이스에 있는 것을 연결해 주는 역할을 하는 것임

미세먼지 앱 --> 공공정보에서 언제든지 사용할 수 있도록 제공해 주는 게 있음. 그거 땡겨와서 내 프로그램으로 만드는 것 (내 프로그램 속에 포함시켜 주는 것이 API)

API 서비스 당겨올 수 있도록 파이썬에서 라이브러리 존재함 (Requests)

공공데이터 활용 사례: 국가에서 제공하는 API 사용해서 (API가 일종의 부품 같은 것임) 부품처럼 프로그래머가 사용할 수 있도록 제공.

개발자에게 프로그래밍의 부품처럼 쓰기 쉽게 표준화해서 알려주는 것이 'request'임

REST API

데이터포맷: JSON+XML이라는 방법으로 groupping함

참고문헌: API 쓸 수 있는 방법(활용가이드)

요청변수: API에다 요청함 (서비스마다 요청변수가 다 다름)

데이터 찾기-데이터 목록에 '미세먼지' 이런 식으로 치면 총 225건이 검색됨 (파일 말고 모든 API) 하면 원하는 것들 찾기

한국환경공단- 전국 미세먼지 다 활용 가능

네이버 API가 유명함 (네이버 오픈 API 목록)- 네이버 로그인, 파파고 번역, 클로바 음성인식...

카카오 REST API- 비즈니스 관련, 인공지능: 카카오 전유물이기 때문에 사용 가능함

In [9]:
#!pip install gradio
#import gradio as gr
#import requests
import pandas as pd

URL 방식으로 주는 것을 'Rest API'라고 함

In [ ]:
requests.get('http://apis.data.go.kr/B552584/ArpltnStatsSvc/''getCtprvnMesureSidoLIst?''sidoName=서울''&searchCondition=hour''&pageNo=1''&numOfRows=100''&returnType=json''&serviceKey=miH%2BZXg85lQy4%2FkmhffvygXDIFiTwisriSLxtyECSzw6hxTVK7yI8QKIOc7mP7LEqCnhBGuj6iW1gZW65dum7A%3D%3D')

In [3]:
URL = ('http://apis.data.go.kr/B552584/ArpltnStatsSvc/' #서비스 URL과 일치
      'getCtprvnMesureSidoLIst?' #구체적인 세부 서비스 이름 (상세기능명 다음에 물음표?)
      'sidoName=서울' #시도이름 (=모양으로)
      '&searchCondition=hour' #&Daily인지, Hour인지 (&로 구분)
      '&pageNo=1' #페이지 넘버 하나
      '&numOfRows=100'
      '&returnType=json' #json의 형태로 받고 싶다
      '&serviceKey=miH%2BZXg85lQy4%2FkmhffvygXDIFiTwisriSLxtyECSzw6hxTVK7yI8QKIOc7mP7LEqCnhBGuj6iW1gZW65dum7A%3D%3D')
response = requests.get(URL)

In [4]:
response

<Response [200]>

다양한 Response Code (python 200: 잘 처리가 됐다는 의미)

#### http://jsonviewer.stack.hu/ (copy json here to view json)

json으로 된 형태가 아래에 담김

In [ ]:
r = response.json()
r

In [6]:
print(type(r))

<class 'dict'>


In [8]:
items = r['response']['body']['items']
items

[{'cityName': '강남구',
  'cityNameEng': 'Gangnam-gu',
  'coValue': '0.3',
  'dataGubun': '',
  'dataTime': '2022-03-27 10:00',
  'districtCode': '02',
  'districtNumSeq': '001',
  'itemCode': '',
  'khaiValue': '',
  'no2Value': '0.017',
  'numOfRows': '10',
  'o3Value': '0.034',
  'pageNo': '1',
  'pm10Value': '33',
  'pm25Value': '11',
  'resultCode': '',
  'resultMsg': '',
  'returnType': 'json',
  'searchCondition': '',
  'serviceKey': '',
  'sidoName': '서울',
  'so2Value': '0.003',
  'totalCount': ''},
 {'cityName': '강동구',
  'cityNameEng': 'Gangdong-gu',
  'coValue': '0.3',
  'dataGubun': '',
  'dataTime': '2022-03-27 10:00',
  'districtCode': '02',
  'districtNumSeq': '002',
  'itemCode': '',
  'khaiValue': '',
  'no2Value': '0.016',
  'numOfRows': '10',
  'o3Value': '0.028',
  'pageNo': '1',
  'pm10Value': '32',
  'pm25Value': '12',
  'resultCode': '',
  'resultMsg': '',
  'returnType': 'json',
  'searchCondition': '',
  'serviceKey': '',
  'sidoName': '서울',
  'so2Value': '0.002',


In [10]:

cityName = [item['cityName'] for item in items] #구에 대한 list
pm10Value = [item['pm10Value'] for item in items] #pm10의 값이 강남구부터 끝까지 리스트로 잡힘
pm25Value = [item['pm25Value'] for item in items]
so2Value = [item['so2Value'] for item in items]
coValue = [item['coValue'] for item in items]
o3Value = [item['o3Value'] for item in items]
no2Value = [item['no2Value'] for item in items]

cityName = pd.Series(cityName) #pd 시리즈: 데이터프레임 속에 들어가는 값들로 변환
pm10Value = pd.Series(pm10Value)   
pm25Value = pd.Series(pm25Value)
so2Value = pd.Series(so2Value)
coValue = pd.Series(coValue)
o3Value = pd.Series(o3Value)
no2Value = pd.Series(no2Value)

df = pd.DataFrame({ 'cityName': cityName, 'pm10Value': pm10Value, 'pm25Value': pm25Value, 'so2Value': so2Value, 'coValue': coValue, 'o3Value': o3Value, 'no2Value': no2Value})

In [11]:
df

,cityName,pm10Value,pm25Value,so2Value,coValue,o3Value,no2Value
0,강남구,33,11,0.003,0.3,0.034,0.017
1,강동구,32,12,0.002,0.3,0.028,0.016
2,강북구,33,11,0.003,0.3,0.039,0.011
3,강서구,34,10,0.003,0.4,0.032,0.022
4,관악구,34,12,0.003,0.4,0.034,0.019
5,광진구,38,13,0.005,0.4,0.033,0.019
6,구로구,37,14,0.003,0.3,0.033,0.015
7,금천구,30,10,0.003,0.3,0.028,0.016
8,노원구,36,14,0.003,0.4,0.032,0.018
9,도봉구,30,17,0.004,0.3,0.037,0.014


도시명: a, b, c, d 있을 때 칼럼에 들어가는 데이터값을 pd 시리즈가 만드는 것임. 

총 7개의 칼럼을 만듦 (pd 시리즈)

데이터프레임이라는 function만 가지고 있으면 cityname, pm10value... 등이 들어감

In [ ]:
def test(sidoName):
  URL = ('http://apis.data.go.kr/B552584/ArpltnStatsSvc/'
      'getCtprvnMesureSidoLIst?'
      'sidoName='+sidoName+
      '&searchCondition=hour'
      '&pageNo=1'
      '&numOfRows=100'
      '&returnType=json'
      '&serviceKey=miH%2BZXg85lQy4%2FkmhffvygXDIFiTwisriSLxtyECSzw6hxTVK7yI8QKIOc7mP7LEqCnhBGuj6iW1gZW65dum7A%3D%3D')
  
  response = requests.get(URL)
  r = response.json()
  items = r['response']['body']['items']

  cityName = [item['cityName'] for item in items]
  pm10Value = [item['pm10Value'] for item in items]
  pm25Value = [item['pm25Value'] for item in items]
  so2Value = [item['so2Value'] for item in items]
  coValue = [item['coValue'] for item in items]
  o3Value = [item['o3Value'] for item in items]
  no2Value = [item['no2Value'] for item in items]

  cityName = pd.Series(cityName)
  pm10Value = pd.Series(pm10Value)   
  pm25Value = pd.Series(pm25Value)
  so2Value = pd.Series(so2Value)
  coValue = pd.Series(coValue)
  o3Value = pd.Series(o3Value)
  no2Value = pd.Series(no2Value)

  df = pd.DataFrame({ 'cityName': cityName, 'pm10Value': pm10Value, 'pm25Value': pm25Value, 'so2Value': so2Value, 'coValue': coValue, 'o3Value': o3Value, 'no2Value': no2Value})
  return df

In [14]:
#!pip install gradio

In [15]:
#import gradio as gr

In [16]:
iface = gr.Interface(test, gr.inputs.Dropdown(['서울', '부산', '대구', '인천', '광주', '대전', '울산', '경기', '강원', '충북', '충남', '전북', '전남', '경북', '경남', '제주', '세종']), outputs=['dataframe'])
iface.launch(debug=True)

NameError: ignored